# Assignment 1

In [6]:
# First, install spacy, pandas, and the language model in the terminal, by typing in:
# pip install spacy pandas
# python -m spacy download en_core_web_md

In [1]:
import spacy
import pandas as pd
import os

/tmp/ipykernel_900/2223173363.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
nlp = spacy.load("en_core_web_md")

In [10]:
# folderpath = os.path.join("../input/USEcorpus/a1")

In [3]:
# create a path to the corpus containing our data and then sort the directory
main_folder_path = "../input/USEcorpus/"
sorted_dir = sorted(os.listdir(main_folder_path))

In [12]:
# with open(folder_path, "r", encoding="latin-1") as file:
    text=file.read()

IndentationError: unexpected indent (1164528371.py, line 2)

In [ ]:
# preprocessing: removing all tokens that simply document metadata or identification

In [4]:
# creating a for loop going into each subfolder in the corpus and opening all text files within the subfolders
for folder in sorted_dir:
    folder_path = os.path.join(main_folder_path, folder)
    filenames = sorted(os.listdir(folder_path))
    for text_file in filenames:
        file_path = folder_path + "/" + text_file
        with open (file_path, encoding="latin-1") as file:
            text = file.read()
       # add text files to a doc object
        doc = nlp(text)

In [23]:
print(doc)

<doc.id=0502.c1>
<title> How the understanding of Wuthering Heights is affected by the structure and the two narrators </title>
In this essay I will discuss the effect with the dual narrators in Wuthering Heights. I will also look at the structure and see what effect that has for the story. Does the book become more interesting with the two narrators, and does the structure contribute to make the story better or does the book become less interesting to read with this construction? First I will discuss the two narrators before I comment on the structure.
	 The main narrator is Mr. Lockwood but within his story Nelly is the narrator. I think Emily Bronte has chosen Mr. Lockwood as the main narrator to make the story more reliable. He is a man and he comes from the upper class while Nelly is a female servant. When the book was written it was more accepted with a male narrator than with a female, and I think that is why Bronte has chosen Mr. Lockwood as her principal narrator.
	 Another re

In [7]:
# create empty list
entities = []

# add each entity to list
for ent in doc.ents:
    entities.append(ent.text)
print(set(entities))

{'L. Cookson', 'Vogler', 'B. Longlerey\n\n', '43', 'Paul', 'Earnshaw', 'Thomas A.', 'Heights', 'Catherine', '27', 'Wuthering Heights', 'America', 'another half-hour', 'Bronte', 'Dean', 'Story and History in Wuthering Heights', '1802', '88', 'Norgate', 'chapter 31', 'Grange', 'Heatcliff', '83', 'eleven', '16', 'First', '248', 'Emily Bronte', 'Nelly', 'Longman Critical Essays, Eds', 'ruff', '139', 'Carl R.', 'The Narrators of Wuthering Heights', 'Woodring', '95', '1801', 'London', '1994', 'two', 'Lockwood', '257'}


In [8]:
# create counters for nouns, verbs, adjectives, and adverbs.
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

# with a for loop, add 1 to the counter every time that part of speech appears in the doc object
for token in doc:
    if token.pos_ == "NOUN":
        noun_count += 1
    elif token.pos_ == "VERB":
        verb_count += 1
    elif token.pos_ == "ADJ":
        adj_count += 1
    elif token.pos_ == "ADV":
        adv_count += 1

In [22]:
# calculating the relative frequency of each part of speech per 10,000 words
relative_freq_noun = (noun_count/len(doc)) * 10000
round(relative_freq_noun, 2)

1378.0

In [23]:
relative_freq_verb = (verb_count/len(doc)) * 10000
round(relative_freq_verb, 2)

1453.86

In [25]:
relative_freq_adj = (adj_count/len(doc)) * 10000
round(relative_freq_adj, 2)

518.33

In [26]:
relative_freq_adv = (adv_count/len(doc)) * 10000
round(relative_freq_adv, 2)

695.32

In [10]:
# extracting named entities for persons, locations, and organizations
persons = set()
for ent in doc.ents:
        if ent.label_ == 'PERSON':
            persons.add(ent.text)

num_persons = len(persons)
print(num_persons)

15


In [29]:
locations = set()
for ent in doc.ents:
        if ent.label_ == 'LOC':
            locations.add(ent.text)

num_locations = len(locations)
print(num_locations)

0


In [30]:
organisations = set()
for ent in doc.ents:
        if ent.label_ == 'ORG':
            organisations.add(ent.text)

num_organisations = len(organisations)
print(num_organisations)

0


In [24]:
annotations = []
for token in doc:
    annotations.append((token.text, token.pos_))

In [25]:
# creating a dataframe with pandas using the spaCy doc
# make one per subfolder
data = pd.DataFrame(annotations, 
                    columns=["Filename", "RelFreq NOUN", "RelFreq VERB", "RelFreq ADJ", "RelFreq ADV", "Unique PER", "Unique LOC", "Unique ORG"])

ValueError: 8 columns passed, passed data had 2 columns

In [15]:
data

,RelFreq NOUNS,RelFreq VERB
0,<,X
1,doc.id=0502.c1,X
2,>,SYM
3,\n,SPACE
4,<,X
...,...,...
1573,"""",PUNCT
1574,\n,SPACE
1575,<,X
1576,/doc,PUNCT


In [ ]:
# upload dataframe to output folder

issues:
- weird tokens? how to preprocess
- takes the last file instead of going through all of them - need to somehow append every file's text to the existing doc object
- cant make data frame with more than two columns
- add in the rest of the tasks to the loop